In [1]:
import ttnn 
import torch
import time 

dim = 2048
num_iters = 100
device = ttnn.open_device(device_id=0)

2025-02-28 12:13:39.630 | DEBUG    | ttnn:<module>:82 - Initial ttnn.CONFIG:
Config{cache_path=/home/bach/.cache/ttnn,model_cache_path=/home/bach/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}
2025-02-28 12:13:40.724 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.pearson_correlation_coefficient be migrated to C++?
2025-02-28 12:13:40.726 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.Conv1d be migrated to C++?
2025-02-28 12:13:40.727 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.conv2d be migrated to C++?
2025-02-28 12:13:40.728 | WARNING  | ttnn.decorators:operation_decorator:801 - Should ttnn.unsq

                 Device | INFO     | Opening user mode device driver

2025-02-28 12:13:40.839 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.30.0, IOMMU: disabled
2025-02-28 12:13:40.840 | INFO     | SiliconDriver   - Detected PCI devices: [0]
2025-02-28 12:13:40.840 | INFO     | SiliconDriver   - Using local chip ids: {0} and remote chip ids {}
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0


## Mapping operations on set of cores

In [5]:
dim = 512
a = torch.randn((dim, dim))
a_t = ttnn.from_torch(a, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)

b = torch.randn((dim, dim))
b_t = ttnn.from_torch(b, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)

d = torch.randn((dim, dim))
d_t = ttnn.from_torch(d, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)

In [6]:
iters = 100

In [9]:
core_grid=ttnn.CoreRangeSet({
            ttnn.CoreRange(ttnn.CoreCoord(0, 0), ttnn.CoreCoord(2, 4)),
        })
core_grid

{[(x=0,y=0) - (x=2,y=4)]}

In [11]:
core_grid=ttnn.CoreGrid(x=2, y=4)
core_grid

ttnn.CoreGrid(x=2, y=4)

In [7]:
## a x b = c -> c x d = e
## grid (4x2) -> (4x2)

start = time.time()

for i in range(iters):
    c_t = ttnn.matmul(
        a_t, 
        b_t,
        core_grid=ttnn.CoreRangeSet({
            ttnn.CoreRange(ttnn.CoreCoord(0, 0), ttnn.CoreCoord(2, 4)),
        })
    )
    
    e_t = ttnn.matmul(
        c_t, 
        d_t,
        core_grid=ttnn.CoreRangeSet({
            ttnn.CoreRange(ttnn.CoreCoord(0, 4), ttnn.CoreCoord(2, 8)),
        })
    )

end = time.time()
print((end - start)/iters)

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: ttnn._ttnn.operations.matmul.matmul_t, input_tensor_a: ttnn._ttnn.tensor.Tensor, input_tensor_b: ttnn._ttnn.tensor.Tensor, *, transpose_a: bool = False, transpose_b: bool = False, memory_config: Optional[ttnn._ttnn.tensor.MemoryConfig] = None, dtype: Optional[ttnn._ttnn.tensor.DataType] = None, program_config: Optional[Union[ttnn::operations::matmul::MatmulMultiCoreProgramConfig, ttnn::operations::matmul::MatmulMultiCoreNonOptimizedReuseProgramConfig, ttnn._ttnn.operations.matmul.MatmulMultiCoreReuseProgramConfig, ttnn._ttnn.operations.matmul.MatmulMultiCoreReuseMultiCastProgramConfig, ttnn._ttnn.operations.matmul.MatmulMultiCoreReuseMultiCast1DProgramConfig, ttnn._ttnn.operations.matmul.MatmulMultiCoreReuseMultiCastDRAMShardedProgramConfig]] = None, activation: Optional[str] = None, compute_kernel_config: Optional[Union[ttnn._ttnn.operations.core.GrayskullComputeKernelConf

In [ ]:
## a x b = c -> c x d = e
## grid (4x4)

start = time.time()

for i in range(iters):
    c_t = ttnn.matmul(
        a_t, 
        b_t,
        core_grid=ttnn.CoreRangeSet({
            ttnn.CoreRange(ttnn.CoreCoord(0, 0), ttnn.CoreCoord(2, 8)),
        })
    )
    
    e_t = ttnn.matmul(
        c_t, 
        d_t,
        core_grid=ttnn.CoreRangeSet({
            ttnn.CoreRange(ttnn.CoreCoord(0, 0), ttnn.CoreCoord(2, 8)),
        })
    )

end = time.time()
print((end - start)/iters)

## L1 vs DRAM

In [ ]:
dim = 8192
a = torch.randn((dim, dim))
a_t = ttnn.from_torch(a, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)

b = torch.randn((dim, dim))
b_t = ttnn.from_torch(b, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)

In [33]:
start = time.time()
for i in range(1000):
    c = ttnn.matmul(
        a_t, 
        b_t,
        dtype=ttnn.bfloat16,
        core_grid=ttnn.CoreGrid(y=8, x=8),
    )
end = time.time()
print("Time: ", end - start)

Time:  38.116708517074585


### from_torch(ROW_MAJOR) -> to_device() = from_torch(ROW_MAJOR, device) 

In [3]:
tot_from_torch_to_dev = 0
tot_from_torch_dev = 0

for i in range(num_iters):

    # from_torch(ROW_MAJOR) -> to_device()
    a = torch.randn((dim, dim)).bfloat16()
    start = time.time()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT)
    a_t = ttnn.to_device(a_t, device=device)
    tot_from_torch_to_dev += time.time() - start
    
    # from_torch(ROW_MAJOR, device)
    a = torch.randn((dim, dim)).bfloat16()
    start = time.time()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    tot_from_torch_dev += time.time() - start
    
tot_from_torch_to_dev /= num_iters
tot_from_torch_dev /= num_iters

print(tot_from_torch_to_dev, tot_from_torch_dev)

0.00044506072998046876 0.00038550615310668947


### tensor on device: tilize() = to_layout() (first time not??)

In [5]:
tot_tilize = 0
tot_to_layout = 0

for i in range(num_iters):
    
    # tilize(DRAM)
    a = torch.randn((dim, dim)).bfloat16()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)
    a_t = ttnn.to_device(a_t, device=device)
    start = time.time()
    a_t = ttnn.tilize(a_t)
    tot_tilize += time.time() - start
    
    # to_layout
    a = torch.randn((dim, dim)).bfloat16()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
    a_t = ttnn.to_device(a_t, device=device)
    start = time.time()
    a_t = ttnn.to_layout(a_t, layout=ttnn.TILE_LAYOUT)
    tot_to_layout += time.time() - start
    
tot_tilize /= num_iters
tot_to_layout /= num_iters

print(tot_tilize, tot_to_layout)

0.0028214049339294433 0.002933976650238037


### tilize(DRAM) = tilize(L1)

In [18]:
tot_tilize_dram = 0
tot_tilize_l1 = 0

for i in range(num_iters):
    
    # tilize(DRAM)
    a = torch.randn((dim, dim)).bfloat16()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)
    a_t = ttnn.to_device(a_t, device=device)
    start = time.time()
    a_t = ttnn.tilize(a_t)
    tot_tilize_dram += time.time() - start
    
    # tilize(L1)
    a = torch.randn((dim, dim)).bfloat16()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
    a_t = ttnn.to_device(a, device=device)
    start = time.time()
    a_t = ttnn.tilize(a_t)
    tot_tilize_l1 += time.time() - start
    
tot_tilize_dram /= num_iters
tot_tilize_l1 /= num_iters

print(tot_tilize_dram, tot_tilize_l1)

0.00423729658126831 0.004347784519195557


### matmul (DRAM) > matmul (L1)

In [11]:
tot_matmul_dram = 0
tot_matmul_l1 = 0

dim = 2048

for i in range(num_iters):
    
    # matmul(DRAM)
    a = torch.randn((dim, dim)).bfloat16()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)
    a_t = ttnn.to_device(a_t, device=device)
    a_t = ttnn.tilize(a_t)

    b = torch.randn((dim, dim)).bfloat16()
    b_t = ttnn.from_torch(b, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.DRAM_MEMORY_CONFIG)
    b_t = ttnn.to_device(b_t, device=device)
    b_t = ttnn.tilize(b_t)

    start = time.time()
    c_t = ttnn.matmul(a_t, b_t, memory_config=ttnn.DRAM_MEMORY_CONFIG)
    tot_matmul_dram += time.time() - start
    
    # matmul(L1)
    a = torch.randn((dim, dim)).bfloat16()
    a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
    a_t = ttnn.to_device(a_t, device=device)
    a_t = ttnn.tilize(a_t)
    
    b = torch.randn((dim, dim)).bfloat16()
    b_t = ttnn.from_torch(b, layout=ttnn.ROW_MAJOR_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
    b_t = ttnn.to_device(b_t, device=device)
    b_t = ttnn.tilize(b_t)

    start = time.time()
    c_t = ttnn.matmul(a_t, b_t, memory_config=ttnn.L1_MEMORY_CONFIG)
    tot_matmul_l1 += time.time() - start
    
tot_matmul_dram /= num_iters
tot_matmul_l1 /= num_iters

print(tot_matmul_dram, tot_matmul_l1)

0.0063421630859375 0.005454301834106445


### Tilize on device <<< on host, but dosn't work with sharded memory configs

In [ ]:
a = torch.randn((dim, dim)).bfloat16()

in0_memory_config = ttnn.create_sharded_memory_config(
    (1, 1, dim, dim),
    core_grid=ttnn.CoreGrid(y=8, x=8),
    strategy=ttnn.ShardStrategy.BLOCK,
    orientation=ttnn.ShardOrientation.ROW_MAJOR,
)
dtype=ttnn.DataType.BFLOAT16

c= ttnn.from_torch(
    a,
    tile=ttnn.Tile((32, 32)),
    # dtype=dtype,
    layout=ttnn.ROW_MAJOR_LAYOUT,
    device=device,
    # memory_config=in0_memory_config
)


c = ttnn.to_layout(c, layout=ttnn.TILE_LAYOUT)#, memory_config = in0_memory_config)
# d = ttnn.tilize(c)
# c

In [21]:
tot_tilize_host = 0
tot_tilize = 0
tot_move = 0
for i in range(num_iters):
    a = torch.randn((dim, dim)).bfloat16()

    start = time.time()
    c = ttnn.from_torch(a, layout=ttnn.TILE_LAYOUT, device=device)
    tot_tilize_host = time.time() - start
    
    a = torch.randn((dim, dim)).bfloat16()

    start = time.time()
    c = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
    tot_move = time.time() - start
    
    start = time.time()
    # d = ttnn.tilize(c)
    d = ttnn.to_layout(c, device=device, memory_config=in0_memory_config)
    tot_tilize = time.time() - start

tot_tilize_host /= num_iters
tot_tilize /= num_iters
tot_move /= num_iters

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: ttnn._ttnn.operations.core.to_layout_t, tensor: ttnn._ttnn.tensor.Tensor, layout: ttnn._ttnn.tensor.Layout, dtype: Optional[ttnn._ttnn.tensor.DataType] = None, memory_config: Optional[ttnn._ttnn.tensor.MemoryConfig] = None, device: ttnn._ttnn.device.Device = None) -> ttnn._ttnn.tensor.Tensor
    2. (self: ttnn._ttnn.operations.core.to_layout_t, tensor: ttnn._ttnn.tensor.Tensor, layout: ttnn._ttnn.tensor.Layout, dtype: Optional[ttnn._ttnn.tensor.DataType] = None, memory_config: Optional[ttnn._ttnn.tensor.MemoryConfig] = None, device: ttnn._ttnn.multi_device.MeshDevice = None) -> ttnn._ttnn.tensor.Tensor

Invoked with: <ttnn._ttnn.operations.core.to_layout_t object at 0x7fcf98890ef0>, ttnn.Tensor([[ 0.27930, -0.09619,  ..., -0.24805,  0.15820],
             [-0.38867,  0.91797,  ...,  2.59375, -0.43359],
             ...,
             [-0.20508,  0.45312,  ...,  1.17969,  1.0

In [18]:
tot_tilize + tot_move

0.003919928073883057

In [19]:
tot_tilize + tot_move

0.003919928073883057

In [10]:
tot_tilize_host

0.003970417976379394

In [8]:
a = dict.fromkeys([1])
a

{1: None}

In [42]:
a = torch.randn((dim, dim)).bfloat16()
b = torch.randn((dim, dim)).bfloat16()


start = time.time()
a_t = ttnn.from_torch(a, layout=ttnn.TILE_LAYOUT, device=device)
b_t = ttnn.from_torch(b, layout=ttnn.TILE_LAYOUT, device=device)
c_1 = ttnn.matmul(a_t, b_t)
tot_tilize_host = time.time() - start

start = time.time()
a_t = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
a_t = ttnn.tilize(a_t)
b_t = ttnn.from_torch(b, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
b_t = ttnn.tilize(b_t)
c_2 = ttnn.matmul(a_t, b_t) 
tot_move = time.time() - start


ttnn.Tensor([[73.50000, -39.50000,  ..., -4.96875, 79.00000],
             [-57.25000, 22.62500,  ..., 36.50000, -46.25000],
             ...,
             [-61.75000, 26.12500,  ..., -77.00000, -30.62500],
             [-32.25000, -51.75000,  ..., -27.00000, 115.00000]], shape=Shape([2048, 2048]), dtype=DataType::BFLOAT16, layout=Layout::TILE)
                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Profiler started on device 0


In [45]:
tot_tilize_host, tot_move

(0.791964054107666, 0.011814594268798828)

In [ ]:
a = torch.randn((dim, dim)).bfloat16()
num_iters = 100
start = time.time()
for i in range(num_iters):
    c = ttnn.to_device(ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT), device=device)
t_avg = (time.time() - start)/num_iters
t_avg

0.0024967384338378906

In [ ]:
(dim*dim*16) / (t_avg * 1e9)

27.01391284629646

In [ ]:
start = time.time()
for i in range(num_iters):
    # a = torch.randn((dim, dim)).bfloat16()
    c = ttnn.from_torch(a, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
t_avg = (time.time() - start)/num_iters
t_avg

0.0025472283363342286

## GRID selection

In [47]:
grid_size = (1,1)
dim = 256
in0_shape = (dim, dim)
in1_shape = (dim, dim)
dtype = ttnn.bfloat16
n_exec = 100

In [50]:
tot_time = 0
for i in range(n_exec):
    in0 = torch.ones(in0_shape).bfloat16()
    in1 = torch.randn(in1_shape).bfloat16()

    in0_t =  ttnn.from_torch(
            in0,
            tile=ttnn.Tile((32, 32)),
            dtype=dtype,
            layout=ttnn.ROW_MAJOR_LAYOUT)
    in0_t = ttnn.to_device(
            in0_t, 
            device=device,
            memory_config=ttnn.DRAM_MEMORY_CONFIG)
    in0_t = ttnn.tilize(in0_t)

    in1_t = ttnn.from_torch(
            in1,
            tile=ttnn.Tile((32, 32)),
            dtype=dtype,
            layout=ttnn.ROW_MAJOR_LAYOUT)
    in1_t = ttnn.to_device(
            in1_t, 
            device=device,
            memory_config=ttnn.DRAM_MEMORY_CONFIG)
    in1_t = ttnn.tilize(in1_t)
        
    start = time.time()
    in2_t = ttnn.matmul(
        in0_t,
        in1_t, 
        core_grid=ttnn.CoreGrid(y=grid_size[1], x=grid_size[0]),
    )
    tot_time += time.time() - start

t_avg = tot_time/n_exec
print(t_avg)

0.0044387316703796385
